<a href="https://colab.research.google.com/github/ask-1710/ABUSIVE-LANGUAGE-DETECTION-IN-URDU/blob/main/Final_copy_UrduFireThreat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install urduhack &> /dev/null
!pip install gensim &> /dev/null
!pip install -U sentence-transformers &> /dev/null
!pip install --upgrade gensim & />dev/null


/bin/bash: dev/null: No such file or directory


In [ ]:
import urduhack
from urduhack import preprocessing as upr
from sklearn import svm  
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score,cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_DIR = '/content/drive/MyDrive/Datasets/HASOC-Urdhu/Threat'
TRAIN_DIR = BASE_DIR+'/train.csv'
TEST_DIR = BASE_DIR+'/test.csv'
test_data = pd.read_csv(TEST_DIR)
train_data = pd.read_csv(TRAIN_DIR)
ORIG_DIR = BASE_DIR+'/test_answ_and_split.csv'
ORIG_data = pd.read_csv(ORIG_DIR)
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def testclassic(model):
  ORIG_DIR = BASE_DIR+'/test_answ_and_split.csv'
  ORIG_data = pd.read_csv(ORIG_DIR)
  df=pd.read_csv(TEST_DIR)
  df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
  vk=vectorizer.transform(df['lemmatized'])
  result=model.predict(vk)
  actual_result=ORIG_data.target
  splitting=zip(result,actual_result,ORIG_data.split)
  private_pred=[]
  private_actual=[]
  public_actual=[]
  public_pred=[]
  for i,j,k in splitting:
    if(k=='public'):
      public_pred.append(i)
      public_actual.append(j)
    else:
      private_pred.append(i)
      private_actual.append(j)

  print("private F1-score",f1_score(private_actual, private_pred, average='weighted'))
  print("private ROC_AUC_score",roc_auc_score(private_actual, private_pred))
  print("public F1-score",f1_score(public_actual, public_pred, average='weighted'))
  print("public ROC_AUC_score",roc_auc_score(public_actual, public_pred))

def trainclassic(model):
  ORIG_data = pd.read_csv(TRAIN_DIR)
  df=pd.read_csv(TRAIN_DIR)
  df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
  vk=vectorizer.transform(df['lemmatized'])
  result_pred=model.predict(vk)
  actual_result=ORIG_data.target
  print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
  print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))


def trainclassicstestplit(model,testX,testY):
  testX = testX.apply(lambda x: lemitizeStr(x))
  vk=vectorizer.transform(df['lemmatized'])
  result_pred=model.predict(vk)
  actual_result=testY
  print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
  print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))
print(ORIG_data['target'].value_counts())
#print(np.unique(result,return_counts=True))

0    3231
1     719
Name: target, dtype: int64


In [ ]:
train_words = set()
for snt in train_data['tweet']:
  for word in snt.split(" "):
    train_words.add(word)
test_words=set()
for snt in test_data['tweet']:
  for word in snt.split(" "):
    test_words.add(word)
new_words=[]
for i in test_words:
  if(i not in train_words):
    new_words.append(i)
print("Train_set new words",len(train_words))
print("Test_set new words",len(test_words))
print("New:  words",len(new_words))
# remove accent
# normalize
#remove punctuation
# sentencetonekinzer
# from urduhack.normalization import normalize_characters
# from urduhack.tokenization import sentence_tokenizer
# from urduhack.preprocessing import remove_punctuation
# def method1(sentence):
#   sentence= upr.remove_accents(sentence)
#   normalized_text = upr.normalize_whitespace(sentence) #space normalized
#   normalized_text_char = normalize_characters(normalized_text)
#   sentences = sentence_tokenizer(normalized_text_char)
#   output=[]
#   for i in sentences:
#     output.append(remove_punctuation(i))
#   return output

# TRAIN=[]
# RESULT=[]
# for i,target in zip(train_data['tweet'],train_data['target']):
#   for j in method1(i):
#     TRAIN.append(j)
#    RESULT.append(target)

Train_set new words 12524
Test_set new words 9947
New:  words 4431


In [ ]:
from urduhack.models.lemmatizer import lemmatizer
from sklearn.model_selection import train_test_split
urduhack.download()

def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str
train_data['lemmatized'] = pd.DataFrame(train_data['tweet'].apply(lambda x: lemitizeStr(x)))
X_train, X_test, Y_train, Y_test = train_test_split(train_data['lemmatized'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
#X_train, X_test, Y_train, Y_test = train_test_split(TRAIN,RESULT, test_size=0.1, random_state=40, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 5000

vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs = vectorizer.fit_transform(X_train)
#test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).transform(X_test)
test_vecs=vectorizer.transform(X_test)
# train_vecs=X_train
# test_vecs=X_test


# Training
SVM = svm.SVC(C=10,kernel='poly',max_iter=2000,gamma='scale',degree=10,probability=True)
SVM=svm.NuSVC(nu=0.3,kernel='rbf',degree=10,gamma='scale',probability=True)
SVM.fit(train_vecs, Y_train)
res = cross_validate(SVM, train_vecs, Y_train, cv=5,  return_train_score=True) 
scores = cross_val_score(SVM,train_vecs, Y_train, cv=5)  
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
#print("Accuracy",SVM.score(test_vecs, Y_test))
# Testing
test_predictionSVM = SVM.predict(test_vecs)
print("NuSVC- rbf, 0.3")
testclassic(SVM)
print("Training")
trainclassic(SVM)
print("AFter split")
trainclassicstestplit(SVM,X_test,Y_test)

In [ ]:
#KNN
from urduhack.models.lemmatizer import lemmatizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors  import KNeighborsClassifier as KNN
def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str
train_data['lemmatized'] = pd.DataFrame(train_data['tweet'].apply(lambda x: lemitizeStr(x)))
X_train, X_test, Y_train, Y_test = train_test_split(train_data['lemmatized'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
#X_train, X_test, Y_train, Y_test = train_test_split(TRAIN,RESULT, test_size=0.1, random_state=40, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 5000

vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs = vectorizer.fit_transform(X_train)
#test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).transform(X_test)
test_vecs=vectorizer.transform(X_test)
# train_vecs=X_train
# test_vecs=X_test


# Training
#SVM = svm.SVC(C=10,kernel='poly',max_iter=2000,gamma='scale',degree=10,probability=True)
KNN=KNN(n_neighbors=7)
KNN.fit(train_vecs, Y_train)
res = cross_validate(SVM, train_vecs, Y_train, cv=5,  return_train_score=True) 
scores = cross_val_score(SVM,train_vecs, Y_train, cv=5)  
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
#print("Accuracy",SVM.score(test_vecs, Y_test))
# Testing
print("KNN,")
testclassic(KNN)
print("TRAINING")
trainclassic(KNN)
#KNN_classifier(train_vecs, Y_train, test_vecs, Y_test)


In [ ]:
urduhack.download()

In [ ]:
from urduhack.models.lemmatizer import lemmatizer
from sklearn.model_selection import train_test_split
urduhack.download()

def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str
train_data['lemmatized'] = pd.DataFrame(train_data['tweet'].apply(lambda x: lemitizeStr(x)))
X_train, X_test, Y_train, Y_test = train_test_split(train_data['lemmatized'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
#X_train, X_test, Y_train, Y_test = train_test_split(TRAIN,RESULT, test_size=0.1, random_state=40, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 5000
vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs = vectorizer.fit_transform(X_train)
#test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).transform(X_test)
test_vecs=vectorizer.transform(X_test)
# train_vecs=X_train
# test_vecs=X_test

def SVM_classifier(train_vecs, Y_train, test_vecs, Y_test):
    # Training
    SVM = svm.SVC(C=10,kernel='poly',max_iter=2000,gamma='scale',degree=10,probability=True)
    #SVM=svm.NuSVC(nu=0.3,kernel='rbf',degree=10,gamma='scale',probability=True)
    SVM.fit(train_vecs, Y_train)
    res = cross_validate(SVM, train_vecs, Y_train, cv=5,  return_train_score=True) 
    scores = cross_val_score(SVM,train_vecs, Y_train, cv=5)  
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    #print("Accuracy",SVM.score(test_vecs, Y_test))
    # Testing
    test_predictionSVM = SVM.predict(test_vecs)
    print("NuSVC- rbf, 0.3")
    testclassic(SVM)

SVM_classifier(train_vecs, Y_train, test_vecs, Y_test)


98304/89078 [=================================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_

Accuracy: 0.82 (+/- 0.00)
NuSVC- rbf, 0.3
private F1-score 0.7301996740728757
private ROC_AUC_score 0.5060295665889933
public F1-score 0.7682849374660142
public ROC_AUC_score 0.4982185273159145


In [ ]:
#LR
from urduhack.models.lemmatizer import lemmatizer
from sklearn.model_selection import train_test_split
urduhack.download()

def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str
train_data['lemmatized'] = pd.DataFrame(train_data['tweet'].apply(lambda x: lemitizeStr(x)))
X_train, X_test, Y_train, Y_test = train_test_split(train_data['lemmatized'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
#X_train, X_test, Y_train, Y_test = train_test_split(TRAIN,RESULT, test_size=0.1, random_state=40, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 5000

vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1,10),  lowercase=True, max_df=50, analyzer='char')
train_vecs = vectorizer.fit_transform(X_train)
#test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).transform(X_test)
test_vecs=vectorizer.transform(X_test)
# train_vecs=X_train
# test_vecs=X_test


# Training
#SVM = svm.SVC(C=10,kernel='poly',max_iter=2000,gamma='scale',degree=10,probability=True)
LR=LogisticRegression(random_state=0)
LR.fit(train_vecs, Y_train)
res = cross_validate(LR, train_vecs, Y_train, cv=5,  return_train_score=True) 
scores = cross_val_score(LR,train_vecs, Y_train, cv=5)  
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
#print("Accuracy",SVM.score(test_vecs, Y_test))
# Testing
test_predictionSVM = LR.predict(test_vecs)
print("Logistic regression")
testclassic(LR)
print("TRAINING")
trainclassic(LR)

Accuracy: 0.83 (+/- 0.00)
Logistic regression
private F1-score 0.7601373567022888
private ROC_AUC_score 0.5423151565889113
public F1-score 0.8155763027388568
public ROC_AUC_score 0.5678387804804715
TRAINING
 F1-score 0.7925654048840655
private ROC_AUC_score 0.5677963212065106


In [ ]:
def savertest1(model):
  df=pd.read_csv(TEST_DIR)
  df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
  vk=vectorizer.transform(df['lemmatized'])
  result=model.predict_proba(vk)
  df['target']=[0 for _ in range(len(result))]
  df['score']=[arr[0] for arr in result]
  print(df.columns)
  df.drop(['tweet','lemmatized'],axis=1).to_csv(BASE_DIR+"/modelSVMpoly83.csv",index = False)
savertest1(model)

Index(['id', 'tweet', 'lemmatized', 'target', 'score'], dtype='object')


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score,cross_validate
def astype_inplace(df: pd.DataFrame, dct):
    df[list(dct.keys())] = df.astype(dct)[list(dct.keys())]
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
X_train1, X_test, Y_train, Y_test = train_test_split(train_data['tweet'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
# df['tweet'].astype('str',inplace=True)
X_train =X_train1.values.tolist()
X_train=model.encode(X_train, batch_size=32,show_progress_bar=True)
y_train = Y_train  
clf = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)
clf.fit(X_train,y_train)
scores = cross_val_score(clf,X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
# ORIG_DIR = BASE_DIR+'/test_answ_and_split.csv'
# ORIG_data = pd.read_csv(ORIG_DIR)
# df=pd.read_csv(TEST_DIR)
# df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
# vk=model.encode(df['lemmatized'], batch_size=32,show_progress_bar=True)
# result=clf.predict(vk)
# actual_result=ORIG_data.target
# splitting=zip(result,actual_result,ORIG_data.split)
# private_pred=[]
# private_actual=[]
# public_actual=[]
# public_pred=[]
# for i,j,k in splitting:
#   if(k=='public'):
#     public_pred.append(i)
#     public_actual.append(j)
#   else:
#     private_pred.append(i)
#     private_actual.append(j)

# print("private F1-score",f1_score(private_actual, private_pred, average='weighted'))
# print("private ROC_AUC_score",roc_auc_score(private_actual, private_pred))


# print("public F1-score",f1_score(public_actual, public_pred, average='weighted'))
# print("public ROC_AUC_score",roc_auc_score(public_actual, public_pred))

ORIG_data = pd.read_csv(TRAIN_DIR)
df=pd.read_csv(TRAIN_DIR)
df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
vk=model.encode(df['lemmatized'], batch_size=32,show_progress_bar=True)
result_pred=clf.predict(vk)
actual_result=ORIG_data.target
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))


In [ ]:
def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    return lemme_str
ORIG_data = pd.read_csv(TRAIN_DIR)
df=pd.read_csv(TRAIN_DIR)
df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
vk=model.encode(df['lemmatized'],batch_size=32,show_progress_bar=True)
result_pred=clf.predict(vk)
actual_result=ORIG_data.target
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))


Batches:   0%|          | 0/188 [00:00<?, ?it/s]

 F1-score 0.9175611385881783
private ROC_AUC_score 0.8420097598787943


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score,cross_validate
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
X_train1, X_test, Y_train, Y_test = train_test_split(train_data['tweet'],train_data['target'], test_size=0.4, random_state=40, shuffle=True)
# df['tweet'].astype('str',inplace=True)
X_train =X_train1.values.tolist()
X_train=model.encode(X_train, batch_size=32,show_progress_bar=True)
y_train = Y_train  
clf = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)
clf.fit(X_train,y_train)
scores = cross_val_score(clf,X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
# ORIG_DIR = BASE_DIR+'/test_answ_and_split.csv'
# ORIG_data = pd.read_csv(ORIG_DIR)
# df=pd.read_csv(TEST_DIR)
# df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
# vk=model.encode(df['lemmatized'], batch_size=32,show_progress_bar=True)
# result=clf.predict(vk)
# actual_result=ORIG_data.target
# splitting=zip(result,actual_result,ORIG_data.split)
# private_pred=[]
# private_actual=[]
# public_actual=[]
# public_pred=[]
# for i,j,k in splitting:
#   if(k=='public'):
#     public_pred.append(i)
#     public_actual.append(j)
#   else:
#     private_pred.append(i)
#     private_actual.append(j)

# print("private F1-score",f1_score(private_actual, private_pred, average='weighted'))
# print("private ROC_AUC_score",roc_auc_score(private_actual, private_pred))


# print("public F1-score",f1_score(public_actual, public_pred, average='weighted'))
# print("public ROC_AUC_score",roc_auc_score(public_actual, public_pred))

ORIG_data = pd.read_csv(TRAIN_DIR)
df=pd.read_csv(TRAIN_DIR)
df['lemmatized'] = df['tweet'].apply(lambda x: lemitizeStr(x))
vk=model.encode(df['lemmatized'],batch_size=32,show_progress_bar=True)
result_pred=clf.predict(vk)
actual_result=ORIG_data.target
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))



Batches:   0%|          | 0/113 [00:00<?, ?it/s]

Accuracy: 0.82 (+/- 0.02)


Batches:   0%|          | 0/188 [00:00<?, ?it/s]

 F1-score 0.9264224508811604
private ROC_AUC_score 0.8516085273630654
